In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import time
import os
import random
import math
import pyautogui
import pyperclip
import tkinter as tk
from tkinter import messagebox, filedialog
import pandas as pd

date = time.strftime("%Y%m%d")

# Excel 파일에서 키워드 불러오기
excel_file_path = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list.xlsx"
df = pd.read_excel(excel_file_path)
keywords = df['title'].dropna().tolist()

font_path='/Users/a/Desktop/font/나눔 글꼴/나눔고딕/NanumFontSetup_TTF_GOTHIC/NanumGothicExtraBold.ttf'
image_final_folder = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated"

# 버튼 이미지 변수 정의
check_emoji =""
hand_emoji=" 바로가기 ▶"

def generate_banner_image_with_text(text, font_path, banner_save_folder, filename):
    # 정방형 이미지로 변경 (1000x1000)
    width, height = 1000, 1000
    font_size = 120  # 정방형에 맞게 폰트 크기 조정

    # 폰트 로드
    font = ImageFont.truetype(font_path, size=font_size)
    
    # 채도가 낮춰진 그라데이션 색상 조합
    soft_gradients = [
        [(220, 150, 150), (255, 255, 255)],  # 부드러운 분홍 → 흰색
        [(150, 180, 230), (255, 255, 255)],  # 부드러운 파랑 → 흰색
        [(230, 200, 150), (255, 255, 255)],  # 부드러운 주황 → 흰색
        [(170, 215, 170), (255, 255, 255)],  # 부드러운 연두 → 흰색
        [(190, 170, 220), (255, 255, 255)],  # 부드러운 보라 → 흰색
        [(220, 170, 200), (255, 255, 255)],  # 부드러운 핑크 → 흰색
        [(150, 190, 200), (255, 255, 255)],  # 부드러운 옥색 → 흰색
    ]
    
    # 랜덤한 그라데이션 색상 선택
    gradient_colors = random.choice(soft_gradients)
    start_color, end_color = gradient_colors
    
    # 빈 이미지 생성
    image = Image.new("RGB", (width, height))
    draw = ImageDraw.Draw(image)
    
    # 랜덤 그라데이션 방향 선택
    direction = random.choice(['left_to_right', 'right_to_left', 'top_to_bottom', 'bottom_to_top', 'diagonal_tl', 'diagonal_tr'])
    
    # 그라데이션 생성 (다양한 방향)
    for y in range(height):
        for x in range(width):
            # 방향에 따른 비율 계산 (0.0 ~ 1.0)
            if direction == 'left_to_right':
                ratio = x / width
            elif direction == 'right_to_left':
                ratio = 1 - (x / width)
            elif direction == 'top_to_bottom':
                ratio = y / height
            elif direction == 'bottom_to_top':
                ratio = 1 - (y / height)
            elif direction == 'diagonal_tl':  # 왼쪽 상단에서 오른쪽 하단
                ratio = (x + y) / (width + height)
            else:  # diagonal_tr: 오른쪽 상단에서 왼쪽 하단
                ratio = (width - x + y) / (width + height)
            
            # 부드러운 변화를 위해 지수 조정 (1.3으로 낮춤)
            ratio = pow(ratio, 1.3)  # 더 부드러운 전환을 위해 지수 감소
            
            # 색상 보간
            r = int(start_color[0] * (1 - ratio) + end_color[0] * ratio)
            g = int(start_color[1] * (1 - ratio) + end_color[1] * ratio)
            b = int(start_color[2] * (1 - ratio) + end_color[2] * ratio)
            
            # 픽셀 색상 설정
            image.putpixel((x, y), (r, g, b))
    
    # 텍스트를 3단어씩 줄바꿈
    words = text.split()
    lines = []
    
    # 3단어씩 그룹화
    for i in range(0, len(words), 3):
        line = ' '.join(words[i:i+3])
        lines.append(line)
    
    # 줄바꿈된 텍스트
    formatted_text = '\n'.join(lines)
    
    # 텍스트 박스 계산
    textbbox = draw.multiline_textbbox((0, 0), formatted_text, font=font, spacing=10)
    
    # 텍스트 그리기 위치 계산
    text_x = (width - textbbox[2] - textbbox[0]) / 2
    text_y = (height - textbbox[3] - textbbox[1]) / 2

    # 더 진한 회색 색상 정의 (더 선명하게)
    dark_gray_color = (30, 30, 30)  # 더 진한 회색
    
    # 텍스트에 테두리 효과 추가 (더 선명하게 보이도록)
    # 텍스트 그림자 효과 (약간의 오프셋으로 그림자 추가)
    offset = 3
    shadow_color = (220, 220, 220)
    draw.multiline_text((text_x+offset, text_y+offset), formatted_text, font=font, fill=shadow_color, align="center", spacing=80)

    # 텍스트 그리기
    draw.multiline_text((text_x, text_y), formatted_text, font=font, fill=(dark_gray_color), align="center", spacing=80)

    # 선명도 향상 (적당히)
    image = image.filter(ImageFilter.SHARPEN)
    
    # 이미지 파일명에 그라데이션 방향 정보 추가
    save_path = os.path.join(banner_save_folder, f"{filename}")
    image.save(save_path, quality=95, dpi=(300, 300))
    print(f"생성 완료: {save_path}")
    return save_path

# 폴더 생성
os.makedirs(image_final_folder, exist_ok=True)

# 각 키워드에 대해 이미지 1개씩 생성
for keyword in keywords:
    # 키워드별 폴더 생성
    keyword_folder = os.path.join(image_final_folder, keyword)
    os.makedirs(keyword_folder, exist_ok=True)
    
    # 배너 이미지 생성 (각 키워드당 1개만)
    filename = f"banner_{keyword}.png"
    generate_banner_image_with_text(keyword, font_path, keyword_folder, filename)

print("모든 키워드에 대한 이미지 생성 완료")

생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/5 월 주일 대표 기도문/banner_5 월 주일 대표 기도문.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/5월 셋째 주일 대표 기도문/banner_5월 셋째 주일 대표 기도문.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/5 월 넷째 주 대표 기도문 모음/banner_5 월 넷째 주 대표 기도문 모음.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/6월 첫째 주일 대표 기도문/banner_6월 첫째 주일 대표 기도문.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/6월 둘째 주 대표 기도문 모음 (2025년)/banner_6월 둘째 주 대표 기도문 모음 (2025년).png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/6월 셋째 주일 대표 기도문 모음 (2025년)/banner_6월 셋째 주일 대표 기도문 모음 (2025년).png
모든 키워드에 대한 이미지 생성 완료
